# <font color = 'green'> Nearest relation workbook - Geospatial line data: GB Only 

#### Documentation: Notebook assessing referenced location data to UK Postcode data
   - Distance to ref line from point
   - Referenced index
   - Referenced indicator (i.e. river type etc)
   - Count of referenced locations within specified radius of postcode centroid
   
#### Author: Alex Armstrong 
#### Created: 06/2022
#### Version: 0.2

## <font color = 'green'> Set up workbook

In [1]:
!python -m pip install adlfs
!pip install geopandas
!pip install adlfs
!pip install pygeos
!pip install rtree

In [2]:
import pandas as pd
import numpy as np
import os
import geopandas as gpd
from shapely.geometry import Point
from zipfile import ZipFile
import matplotlib.pyplot as plt

In [3]:
from AZURE_VARS import *

## <font color = 'green'> Import ONSPD data

In [4]:
usecols = ['pcds', 'doterm', 'oseast1m', 'osnrth1m', 'ctry', 'ru11ind']

- pcds = Unit postcode variable length
- doterm = date of termination, if __= null__ then postcode is live
- oseast1m = National grid reference - Easting
- oseast1m = National grid reference - Northing
- ctry = Country
- ru11ind = 2011 Census rural-urban classification 

In [5]:
onspd = pd.read_csv('onspd.csv', usecols=usecols, low_memory=False)
onspd

,pcds,doterm,oseast1m,osnrth1m,ctry,ru11ind
0,AB1 0AA,199606.0,385386.0,801193.0,S92000003,3
1,AB1 0AB,199606.0,385177.0,801314.0,S92000003,3
2,AB1 0AD,199606.0,385053.0,801092.0,S92000003,3
3,AB1 0AE,199606.0,384600.0,799300.0,S92000003,6
4,AB1 0AF,199207.0,384460.0,800660.0,S92000003,3
...,...,...,...,...,...,...
2673013,ZE3 9JW,NaN,438975.0,1110038.0,S92000003,8
2673014,ZE3 9JX,NaN,438872.0,1110219.0,S92000003,8
2673015,ZE3 9JY,NaN,438498.0,1112029.0,S92000003,8
2673016,ZE3 9JZ,NaN,438662.0,1112122.0,S92000003,8


<font color=blue> Get point objects and convert to geo-dataframe

In [6]:
coords = [Point(xy) for xy in zip(onspd.oseast1m.values, onspd.osnrth1m.values)]
geo_pcu = gpd.GeoDataFrame(onspd, geometry=coords, crs='EPSG:27700')

In [7]:
geo_pcu

,pcds,doterm,oseast1m,osnrth1m,ctry,ru11ind,geometry
0,AB1 0AA,199606.0,385386.0,801193.0,S92000003,3,POINT (385386.000 801193.000)
1,AB1 0AB,199606.0,385177.0,801314.0,S92000003,3,POINT (385177.000 801314.000)
2,AB1 0AD,199606.0,385053.0,801092.0,S92000003,3,POINT (385053.000 801092.000)
3,AB1 0AE,199606.0,384600.0,799300.0,S92000003,6,POINT (384600.000 799300.000)
4,AB1 0AF,199207.0,384460.0,800660.0,S92000003,3,POINT (384460.000 800660.000)
...,...,...,...,...,...,...,...
2673013,ZE3 9JW,NaN,438975.0,1110038.0,S92000003,8,POINT (438975.000 1110038.000)
2673014,ZE3 9JX,NaN,438872.0,1110219.0,S92000003,8,POINT (438872.000 1110219.000)
2673015,ZE3 9JY,NaN,438498.0,1112029.0,S92000003,8,POINT (438498.000 1112029.000)
2673016,ZE3 9JZ,NaN,438662.0,1112122.0,S92000003,8,POINT (438662.000 1112122.000)


In [8]:
geo_pcu.crs

<Derived Projected CRS: EPSG:27700>
Name: OSGB36 / British National Grid
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: United Kingdom (UK) - offshore to boundary of UKCS within 49°45'N to 61°N and 9°W to 2°E; onshore Great Britain (England, Wales and Scotland). Isle of Man onshore.
- bounds: (-9.0, 49.75, 2.01, 61.01)
Coordinate Operation:
- name: British National Grid
- method: Transverse Mercator
Datum: Ordnance Survey of Great Britain 1936
- Ellipsoid: Airy 1830
- Prime Meridian: Greenwich

<font color=blue> Remove missing coordinates from data

In [9]:
geo_pcu['oseast1m'].isnull().unique()

array([False,  True])

In [10]:
len(geo_pcu)

2673018

In [11]:
geo_pcu = geo_pcu[geo_pcu['oseast1m'].notna()]

In [12]:
len(geo_pcu)

2649532

<font color=blue> Identify what postcodes are live / terminated using doterm field

In [13]:
live = geo_pcu[geo_pcu.doterm.isna()]
terminated = geo_pcu[~geo_pcu.doterm.isna()]

In [14]:
print(len(live))
print(len(terminated))

1772619
876913


## <font color = 'green'> Import referenced line location data

### Import data from Azure

<font color=blue> Amend BLOBNAME to relevant file path

In [20]:
waterways = gpd.read_file('GB_Waterways.zip')

In [22]:
loc_data_geo = waterways[waterways['form'] != 'lake']

In [23]:
len(loc_data_geo)

165896

### Clean data - reduce to GB only

<font color = 'blue'> Check if ref data includes NI / Channel islands

In [36]:
geo_pcu[geo_pcu.pcds.str.startswith(("BT", "GY", "JE", "IM"))].count()

pcds        0
doterm      0
oseast1m    0
osnrth1m    0
ctry        0
ru11ind     0
geometry    0
dtype: int64

<font color = 'blue'>Remove NI / Channel islands from ONSPD & imported data

In [39]:
geo_pcu['Area'] = geo_pcu['pcds'].str[:2]
geo_pcu = geo_pcu[geo_pcu['Area'] != 'BT']

Count to make sure they were dropped

In [40]:
geo_pcu[geo_pcu.pcds.str.startswith(("BT", "GY", "JE", "IM"))].count()

pcds        0
doterm      0
oseast1m    0
osnrth1m    0
ctry        0
ru11ind     0
geometry    0
Area        0
dtype: int64

Drop 'Area' columns as no longer required

In [41]:
unwanted_cols = ['Area']

In [42]:
geo_pcu = geo_pcu.drop(columns=unwanted_cols)
#loc_data_geo = loc_data_geo.drop(columns=unwanted_cols)

#### <font color = 'red'> Remove terminated postcodes from df if you wish

In [43]:
#geo_pcu = geo_pcu[geo_pcu.doterm.isna()]

### Plot initial map to look at data

In [44]:
# gb_boundary = gpd.read_file('GB_Boundaries_2017.zip') 

In [45]:
# plt.figure()
# fig, ax = plt.gcf(), plt.gca()
# fig.set_size_inches(10, 10)
# loc_data_geo.plot(ax=ax, markersize=0.5, color = 'red')
# #geo_pcu.plot(ax=ax, alpha=0.1, markersize=0.1)
# gb_boundary.boundary.plot(color='black', linewidth=0.5, ax=ax)
# ax.axis('off');

# Nearest line to point

Get centroid for each postcodegeo_pcu

In [36]:
import numpy as np

In [37]:
def min_distance(point):
    """Calculate the distance (meters) from point data (loc_data_point) to nearest line (loc_data_geo)"""
    return loc_data_geo.distance(point).min()

In [38]:
# %%time

# geo_pcu['min_dist_to_lines'] = geo_pcu.head(1000).geometry.apply(min_distance)

In [39]:
%%time

geo_pcu['min_dist_to_lines'] = geo_pcu.geometry.apply(min_distance)

CPU times: user 3d 6h 28min 6s, sys: 9.34 s, total: 3d 6h 28min 15s
Wall time: 3d 6h 28min 17s


In [40]:
geo_pcu.head(5)

,pcds,doterm,oseast1m,osnrth1m,ctry,ru11ind,geometry,min_dist_to_lines
0,AB1 0AA,199606.0,385386.0,801193.0,S92000003,3,POINT (385386.000 801193.000),562.679098
1,AB1 0AB,199606.0,385177.0,801314.0,S92000003,3,POINT (385177.000 801314.000),569.055261
2,AB1 0AD,199606.0,385053.0,801092.0,S92000003,3,POINT (385053.000 801092.000),320.304830
3,AB1 0AE,199606.0,384600.0,799300.0,S92000003,6,POINT (384600.000 799300.000),326.466497
4,AB1 0AF,199207.0,384460.0,800660.0,S92000003,3,POINT (384460.000 800660.000),131.860561


# <font color = 'green'> Count within radius of point

<font color = 'blue'> Create a buffer around point data and count ref values within buffer

In [46]:
# Make an empty DF and add postcodes for reference
Count_df = pd.DataFrame()
Count_df['pcds'] = geo_pcu['pcds']

<font color = 'blue'> Specify buffer size (in meters)

In [47]:
# Amend distance field below to change size of buffer - in meters
distance = 500

<font color = 'blue'> Using defined buffer size create a buffer around each point and set as new gdf

In [48]:
Count_df['geometry'] = geo_pcu.buffer(distance)

In [49]:
Count_df = gpd.GeoDataFrame(Count_df)

In [50]:
Count_df = Count_df.set_geometry('geometry')

<font color = 'blue'> Count referenced location data within specified buffer

In [51]:
Count_df

,pcds,geometry
0,AB1 0AA,"POLYGON ((385886.000 801193.000, 385883.592 80..."
1,AB1 0AB,"POLYGON ((385677.000 801314.000, 385674.592 80..."
2,AB1 0AD,"POLYGON ((385553.000 801092.000, 385550.592 80..."
3,AB1 0AE,"POLYGON ((385100.000 799300.000, 385097.592 79..."
4,AB1 0AF,"POLYGON ((384960.000 800660.000, 384957.592 80..."
...,...,...
2673013,ZE3 9JW,"POLYGON ((439475.000 1110038.000, 439472.592 1..."
2673014,ZE3 9JX,"POLYGON ((439372.000 1110219.000, 439369.592 1..."
2673015,ZE3 9JY,"POLYGON ((438998.000 1112029.000, 438995.592 1..."
2673016,ZE3 9JZ,"POLYGON ((439162.000 1112122.000, 439159.592 1..."


In [52]:
data_merged = gpd.sjoin(Count_df, loc_data_geo, how="inner", predicate='intersects')

In [53]:
data_merged.groupby('pcds').agg('count')

,geometry,index_right,fid,id,fictitious,flowDirect,length,form,watercours,watercou_1,startNode,endNode
pcds,,,,,,,,,,,,
AB1 0AD,4,4,4,4,4,4,4,4,3,0,4,4
AB1 0AE,1,1,1,1,1,1,1,1,1,0,1,1
AB1 0AF,6,6,6,6,6,6,6,6,5,1,6,6
AB1 0AG,5,5,5,5,5,5,5,5,5,1,5,5
AB1 0AJ,4,4,4,4,4,4,4,4,3,0,4,4
...,...,...,...,...,...,...,...,...,...,...,...,...
ZE2 9XX,1,1,1,1,1,1,1,1,1,0,1,1
ZE2 9YB,4,4,4,4,4,4,4,4,2,0,4,4
ZE2 9YD,15,15,15,15,15,15,15,15,8,0,15,15


In [54]:
Count = data_merged.groupby(['pcds','form','watercours'], as_index=False)['index_right'].count()
Count.columns = ['pcds','nearest_form','nearest_watercourse_name','count_ref_loc_within_radius']
Count

,pcds,nearest_form,nearest_watercourse_name,count_ref_loc_within_radius
0,AB1 0AD,inlandRiver,River Dee,3
1,AB1 0AE,inlandRiver,River Dee,1
2,AB1 0AF,inlandRiver,Culter Burn,1
3,AB1 0AF,inlandRiver,River Dee,4
4,AB1 0AG,inlandRiver,Buckler Burn,1
...,...,...,...,...
1441556,ZE2 9YF,inlandRiver,Burn of Droswall,1
1441557,ZE2 9YF,inlandRiver,Burn of Weisdale,2
1441558,ZE2 9ZG,inlandRiver,Burn of Voesgarth,1
1441559,ZE2 9ZG,tidalRiver,Burn of Gerdie,1


<font color = 'blue'> Merge count dataframe onto previously created distance dataframe

In [55]:
ref_data = geo_pcu.merge(Count, on=['pcds','pcds'], how='left')
ref_data

,pcds,doterm,oseast1m,osnrth1m,ctry,ru11ind,geometry,nearest_form,nearest_watercourse_name,count_ref_loc_within_radius
0,AB1 0AA,199606.0,385386.0,801193.0,S92000003,3,POINT (385386.000 801193.000),NaN,NaN,NaN
1,AB1 0AB,199606.0,385177.0,801314.0,S92000003,3,POINT (385177.000 801314.000),NaN,NaN,NaN
2,AB1 0AD,199606.0,385053.0,801092.0,S92000003,3,POINT (385053.000 801092.000),inlandRiver,River Dee,3.0
3,AB1 0AE,199606.0,384600.0,799300.0,S92000003,6,POINT (384600.000 799300.000),inlandRiver,River Dee,1.0
4,AB1 0AF,199207.0,384460.0,800660.0,S92000003,3,POINT (384460.000 800660.000),inlandRiver,Culter Burn,1.0
...,...,...,...,...,...,...,...,...,...,...
3025243,ZE3 9JW,NaN,438975.0,1110038.0,S92000003,8,POINT (438975.000 1110038.000),NaN,NaN,NaN
3025244,ZE3 9JX,NaN,438872.0,1110219.0,S92000003,8,POINT (438872.000 1110219.000),NaN,NaN,NaN
3025245,ZE3 9JY,NaN,438498.0,1112029.0,S92000003,8,POINT (438498.000 1112029.000),NaN,NaN,NaN
3025246,ZE3 9JZ,NaN,438662.0,1112122.0,S92000003,8,POINT (438662.000 1112122.000),NaN,NaN,NaN


## <font color = 'blue'> Total line length within buffer of each point

In [56]:
def line_length(area):
    '''Function to calculate total length of lines within defined buffer around each point'''
    line_within_area = loc_data_geo.geometry.intersection(area)
    return line_within_area.length.sum() / 1000

In [ ]:
%%time
ref_data['line_length'] = Count_df['geometry'].apply(line_length)

In [ ]:
ref_data.to_csv(r'----.csv')

## <font color = 'blue'> Tidy df ready for export

In [54]:
output = ref_data.drop(columns=['oseast1m','osnrth1m','geometry'])
output

,pcds,doterm,ctry,ru11ind,min_dist_to_lines,nearest_form,nearest_watercourse_name,count_ref_loc_within_radius
0,AB1 0AA,199606.0,S92000003,3,562.679098,NaN,NaN,NaN
1,AB1 0AB,199606.0,S92000003,3,569.055261,NaN,NaN,NaN
2,AB1 0AD,199606.0,S92000003,3,320.304830,inlandRiver,River Dee,3.0
3,AB1 0AE,199606.0,S92000003,6,326.466497,inlandRiver,River Dee,1.0
4,AB1 0AF,199207.0,S92000003,3,131.860561,inlandRiver,Culter Burn,1.0
...,...,...,...,...,...,...,...,...
3025243,ZE3 9JW,NaN,S92000003,8,2798.931100,NaN,NaN,NaN
3025244,ZE3 9JX,NaN,S92000003,8,2656.078410,NaN,NaN,NaN
3025245,ZE3 9JY,NaN,S92000003,8,817.621259,NaN,NaN,NaN
3025246,ZE3 9JZ,NaN,S92000003,8,746.191385,NaN,NaN,NaN


# Save to file

In [55]:
#output.to_csv('geo_ref_data.csv', index=False)

In [56]:
output.to_csv(r'----.csv')